In [13]:
import pandas as pd
import numpy as np


df = pd.read_csv('Новая таблица - Лист1.csv')


df['Среднее кол-во людей'].fillna(df['Среднее кол-во людей'].mean(), inplace=True)

def generate_15min_counts(avg_count):
    fifteen_min_data = {}
    for hour in range(24):
        for quarter in range(4):  
            interval = f'Час_{hour}_Минута_{quarter*15}'
            if 6 <= hour <= 9:
                # Утренний пик
                mean = avg_count * 1.5
                std_dev = avg_count * 0.25
            elif 17 <= hour <= 22:
                # Вечерний пик
                mean = avg_count * 1.5
                std_dev = avg_count * 0.25
            else:
                # Вне пиковых часов
                mean = avg_count * 0.5
                std_dev = avg_count * 0.5

            count = int(np.random.normal(mean, std_dev))
            count = max(0, count)  # Убедимся, что значение не отрицательное
            fifteen_min_data[interval] = count
    return fifteen_min_data

fifteen_min_counts = df['Среднее кол-во людей'].apply(generate_15min_counts)
fifteen_min_df = pd.DataFrame(fifteen_min_counts.tolist())

expanded_df = pd.concat([df, fifteen_min_df], axis=1)


(expanded_df.head())

,Индекс остановки,Название остановки,Среднее кол-во людей,Маршрут 1,Маршрут 2,Маршрут 3,Маршрут 4,Маршрут 5,Маршрут 6,Маршрут 7,...,Час_21_Минута_30,Час_21_Минута_45,Час_22_Минута_0,Час_22_Минута_15,Час_22_Минута_30,Час_22_Минута_45,Час_23_Минута_0,Час_23_Минута_15,Час_23_Минута_30,Час_23_Минута_45
0,1,ВДНХ (главный вход),25.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,42,33,26,39,49,39,3,31,0,16
1,2,Метро ВДНХ,30.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,49,41,32,35,45,41,9,0,9,15
2,3,ул Бориса Галушкина,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16,13,21,17,11,18,10,15,5,0
3,4,Ростокинская улица,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,18,26,22,24,22,15,0,4,3,10
4,5,Детский театр,21.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,29,30,43,30,23,25,0,9,17,21


In [11]:
expanded_df.to_csv('expanded_data.csv')

## Training

In [67]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import TensorDataset, DataLoader
df = pd.read_csv('expanded_data_new.csv', delimiter=';')
del df['Unnamed: 0']


In [68]:
df.to_csv('expanded_data_new.csv')

In [56]:
df

,Индекс остановки,Название остановки,Час_0_Минута_0,Час_0_Минута_15,Час_0_Минута_30,Час_0_Минута_45,Час_1_Минута_0,Час_1_Минута_15,Час_1_Минута_30,Час_1_Минута_45,...,Час_21_Минута_30,Час_21_Минута_45,Час_22_Минута_0,Час_22_Минута_15,Час_22_Минута_30,Час_22_Минута_45,Час_23_Минута_0,Час_23_Минута_15,Час_23_Минута_30,Час_23_Минута_45
0,1,ВДНХ (главный вход),27,0,27,27,26,9,0,10,...,34,37,44,26,35,35,0,7,17,27
1,2,Метро ВДНХ,0,7,24,13,6,0,3,18,...,52,51,47,31,58,55,35,5,20,4
2,3,ул Бориса Галушкина,6,5,7,0,10,1,9,16,...,16,19,11,11,10,15,9,17,0,1
3,4,Ростокинская улица,6,15,5,9,3,0,12,2,...,20,22,27,17,14,24,1,8,0,11
4,5,Детский театр,19,9,7,7,0,10,30,0,...,30,38,34,38,33,41,0,10,13,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,МЦК Деловой Центр,14,4,9,15,0,4,9,0,...,39,39,45,43,38,45,12,16,7,9
96,97,Дом правительства,0,2,7,2,9,5,10,5,...,10,9,11,13,9,9,5,1,1,2
97,98,Долгопрудный,11,8,0,10,6,0,20,4,...,20,18,14,15,16,21,0,0,0,2
98,99,Росатом,0,1,0,0,0,0,1,0,...,3,3,2,2,2,2,0,1,0,0


In [57]:

df_numeric = df.iloc[:, 2:]

scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df_numeric)

def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps)]
        Xs.append(v)
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 10
X, y = create_dataset(df_scaled, df_scaled, time_steps)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

batch_size = 64  
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [58]:
print(X[1][0])

[0.         0.08139535 0.4        0.13978495 0.10344828 0.
 0.05555556 0.36734694 0.         0.21621622 0.35       0.13924051
 0.15909091 0.35       0.27536232 0.29545455 0.17567568 0.
 0.01388889 0.15625    0.05172414 0.         0.31818182 0.38888889
 0.20325203 0.25139665 0.31007752 0.37931034 0.33070866 0.40677966
 0.23529412 0.54285714 0.46363636 0.28776978 0.34745763 0.32307692
 0.33858268 0.3852459  0.39837398 0.42016807 0.         0.12941176
 0.03846154 0.43181818 0.31481481 0.         0.03508772 0.38297872
 0.32467532 0.18518519 0.34545455 0.         0.20779221 0.
 0.27142857 0.         0.25333333 0.27118644 0.27927928 0.
 0.         0.4        0.         0.14414414 0.4        0.04761905
 0.23404255 0.04545455 0.22758621 0.3220339  0.35964912 0.25757576
 0.5042735  0.32       0.39230769 0.28947368 0.31578947 0.30215827
 0.368      0.33093525 0.20979021 0.33898305 0.33333333 0.33088235
 0.41803279 0.32231405 0.43697479 0.35915493 0.44339623 0.25203252
 0.44274809 0.47826087 0.30

In [22]:
!pip install scikit-learn

  Using cached scikit_learn-1.3.2-cp38-cp38-macosx_10_9_x86_64.whl (10.1 MB)
  Using cached scipy-1.10.1-cp38-cp38-macosx_10_9_x86_64.whl (35.0 MB)
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)
You should consider upgrading via the '/Users/danilo/14Bit_MPIT_Hack/bin/python -m pip install --upgrade pip' command.


In [59]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        
        out, (hn, cn) = self.lstm(x, (h0, c0))
        
        out = self.fc(out[:, -1, :])
        return out

input_dim = X_train.shape[2]
hidden_dim = 50
num_layers = 2
output_dim = y_train.shape[1]

model = LSTMModel(input_dim, hidden_dim, num_layers, output_dim)

In [60]:
# Обучение модели
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

num_epochs = 1000  # Можно настроить
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        #print(inputs)
        #print(labels)
        optimizer.zero_grad()
        y_pred = model(inputs)
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
    if epoch % 100 == 0:
        
        print(f'Epoch {epoch} Loss: {loss.item()}')

# Оценка модели
def evaluate_model(model, test_loader):
    with torch.no_grad():
        predictions, actuals = [], []
        for inputs, labels in test_loader:
            y_pred = model(inputs)
            predictions.append(y_pred.numpy())
            actuals.append(labels.numpy())
    predictions = np.vstack(predictions)
    #print(predictions)
    #print(actuals)
    actuals = np.vstack(actuals)
    mse = ((predictions - actuals)**2).mean()
    return mse

mse = evaluate_model(model, test_loader)
print(f'Test MSE: {mse}')

Epoch 0 Loss: 0.2066064327955246
Epoch 100 Loss: 0.014096061699092388
Epoch 200 Loss: 0.003047242760658264
Epoch 300 Loss: 0.000992623157799244
Epoch 400 Loss: 0.00048582861199975014
Epoch 500 Loss: 0.00022663323034066707
Epoch 600 Loss: 0.0002532036160118878
Epoch 700 Loss: 0.00010075186582980677
Epoch 800 Loss: 0.00023731880355626345
Epoch 900 Loss: 9.910917287925258e-05
Test MSE: 0.05274975299835205


## Inference

In [65]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler

new_data = df

new_data_processed = new_data.drop(['Индекс остановки', 'Название остановки'], axis=1)

scaler = MinMaxScaler(feature_range=(0, 1))
new_data_scaled = scaler.fit_transform(new_data_processed)

def create_sequences(input_data, time_steps):
    sequences = []
    for i in range(len(input_data) - time_steps):
        sequence = input_data[i:i + time_steps]
        sequences.append(sequence)
    return np.array(sequences)

time_steps = 10 
sequences = create_sequences(new_data_scaled, time_steps)


sequences_tensor = torch.tensor(sequences, dtype=torch.float32)

model.eval()  
with torch.no_grad():
    predictions = model(sequences_tensor)

predictions_numpy = predictions.numpy()
predictions_rescaled = scaler.inverse_transform(predictions_numpy)

predictions_rounded = np.round(predictions_rescaled).astype(int)

print(predictions_rounded)

[[14 29 22 ... 12 17  7]
 [ 2  3  2 ...  2  6  2]
 [ 2 -1  7 ...  8 11  5]
 ...
 [11  7  1 ...  0  0  2]
 [ 1  1  0 ...  0  0  0]
 [ 3 -1  2 ...  1  4  2]]


In [66]:
predictions_rounded[0]

array([14, 29, 22,  6,  7,  1,  9,  0, 10, 19,  5, 16, 22,  1,  2, 29,  5,
       14, 13,  9,  2, 17, 13,  0, 36, 36, 29, 37, 30, 41, 32, 33, 27, 39,
       40, 42, 34, 35, 32, 34,  7, 19,  3, 14, 11,  2,  4,  0,  6, 19,  0,
       10, 20,  2,  0, 18,  6,  0, 15, 13, 13, 28,  6,  5, 10,  9, 29, 11,
       37, 39, 28, 29, 20, 25, 29, 25, 39, 42, 33, 45, 27, 34, 34, 35, 30,
       21, 46, 39, 33, 39, 36, 30,  8, 12, 17,  7])

In [72]:

time_columns = [col for col in df.columns if col.startswith('Час')]

transformed_data = []

for index, row in df.iterrows():
    for time_col in time_columns:
        parts = time_col.split('_')
        hour = int(parts[1])
        minute = int(parts[3])

        transformed_data.append({
            'Индекс остановки': row['Индекс остановки'],
            'Название остановки': row['Название остановки'],
            'Час': hour,
            'Минута': minute,
            'Количество людей': row[time_col]
        })

transformed_df = pd.DataFrame(transformed_data)

for i in range(1, 11):
    transformed_df[f'Количество людей_{i}h_назад'] = transformed_df.groupby('Индекс остановки')['Количество людей'].shift(i * 4)

transformed_df.dropna(inplace=True)

features = transformed_df[[f'Количество людей_{i}h_назад' for i in range(1, 11)]]
target = transformed_df['Количество людей']


In [86]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

import numpy as np

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


catboost_model = CatBoostRegressor(iterations=10000, depth=7, learning_rate=0.01, loss_function='RMSE', verbose=True)
catboost_model.fit(X_train, y_train)

catboost_predictions = catboost_model.predict(X_test)
catboost_mse = mean_squared_error(y_test, catboost_predictions)
catboost_rmse = np.sqrt(catboost_mse)

print("RMSE для CatBoost:", catboost_rmse)


In [81]:

predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)

In [ ]:
transformed_df['Индекс остановки'] = transformed_df['Индекс остановки'].astype('category')

features_with_stop_info = pd.concat([features, transformed_df['Индекс остановки']], axis=1)

X_train_stop, X_test_stop, y_train_stop, y_test_stop = train_test_split(features_with_stop_info, target, test_size=0.2, random_state=42)

model_with_stop_info = RandomForestRegressor(n_estimators=1000, random_state=42)
model_with_stop_info.fit(X_train_stop, y_train_stop)

predictions_with_stop_info = model_with_stop_info.predict(X_test_stop)
mse_with_stop_info = mean_squared_error(y_test_stop, predictions_with_stop_info)
rmse_with_stop_info = np.sqrt(mse_with_stop_info)

rmse_with_stop_info


In [103]:
full_model = RandomForestRegressor(n_estimators=1000, random_state=42, verbose=True)
full_model.fit(features_with_stop_info, target)

import joblib
model_filename = 'random_forest_full_model.joblib'
joblib.dump(full_model, model_filename)

model_filename


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    1.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    3.8s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    8.4s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:   14.9s


'random_forest_full_model.joblib'

In [104]:
import joblib

model = joblib.load('random_forest_full_model.joblib')

predictions = model.predict(X_test_stop)

# Вывод предсказаний
print(predictions)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.2s


[17.415  6.506  3.53  ... 22.203 15.668  3.076]


In [106]:
type(X_test_stop)

pandas.core.frame.DataFrame

In [113]:
features_with_stop_info.to_csv('test_data.csv')

In [ ]:
test_df = pd.read_json('test_data (1).json')
test_df = test_df
del test_df['FIELD1']
int(model.predict(test_df))

In [ ]:
test_df